# GPU enabled Kubeflow notebook
In this notebook we demonstrate multi GPU training using tensorflow framework. In the example we use convolutional neural network for image classification trained on cifar10 dataset. Example was tested on [charmed kubeflow](https://charmed-kubeflow.io/). 

**Important**

This example was tested on drivers:
```
NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7
```
And ```tensorflow-gpu==2.10.0```. Check rest of the requirements in the attatched `requirements.txt`.

In [1]:
# Import libs and setup environment
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Set for optimized memory usage
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
# os.environ["NCCL_DEBUG"] = "WARN"

# Allow to use whole GPU memory
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True

tf.config.experimental.list_physical_devices()

2023-02-17 21:34:24.823880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 21:34:24.982229: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-17 21:34:25.663971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 21:34:25.664143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load d

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [2]:
# Extract GPU device names
device_type = 'GPU'
devices = tf.config.experimental.list_physical_devices(
          device_type)
devices_names = [d.name.split("e:")[1] for d in devices]
devices_names

['GPU:0', 'GPU:1', 'GPU:2', 'GPU:3', 'GPU:4', 'GPU:5', 'GPU:6', 'GPU:7']

In [3]:
# Image recognition dataset for 10 class prediction
# https://www.cs.toronto.edu/~kriz/cifar.html
# Image has 32x32x3 resolution
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [4]:
# Scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [5]:
# One hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [6]:
def get_model():
    '''
    Create simple deep model. 3 hidden layers. 
    '''
    strategy = tf.distribute.MirroredStrategy(
           devices=devices_names)
    with strategy.scope():
        model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])
        model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
def get_model_cnn():
    '''
    Create More sophisticated model. Using mirrored distributed training strategy
    Reading: https://www.tensorflow.org/guide/distributed_training#mirroredstrategy
    '''
    strategy = tf.distribute.MirroredStrategy(
           devices=devices_names)
    with strategy.scope():
        model = keras.models.Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(10, activation='softmax'))
        # compile model
        opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

In [8]:
m = get_model_cnn()

2023-02-17 21:34:40.730487: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 21:34:47.051156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38406 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-02-17 21:34:47.054266: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38406 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2023-02-17 21:34:47.057240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


/opt/conda/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
m.fit()

In [17]:
# MultiGPU prediction. Run nvidia-smi command in another terminal tab to check GPU ussage
batch_size_per_gpu = 32
n_gpus = len(devices_names)
model_gpu = get_model_cnn()
model_gpu.fit(X_train_scaled, y_train_encoded, epochs=100, batch_size=batch_size_per_gpu * n_gpus)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Epoch 1/100
196/196 [==============================] - 8s 8ms/step - loss: 1.9678 - accuracy: 0.2979
Epoch 2/100
196/196 [==============================] - 2s 11ms/step - loss: 1.7502 - accuracy: 0.3866
Epoch 3/100
196/196 [==============================] - 1s 7ms/step - loss: 1.6521 - accuracy: 0.4193
Epoch 4/100
196/196 [==============================] - 2s 12ms/step - loss: 1.5554 - accuracy: 0.4527
Epoch 5/100
196/196 [==============================] - 1s 7ms/step - loss: 1.4887 - accuracy: 0.4764
Epoch 6/100
196/196 [==============================] - 2

In [ ]:
# CPU only prediction
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 1)

In [9]:
# Single GPU example
%%timeit -n1 -r1
# GPU
with tf.device('GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled[:500], y_train_encoded[:500], epochs = 10)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2022-10-06 09:31:07.019013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 09:31:08.332506: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2022-10-06 09:31:08.332701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 216 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:34:00.0, compute capability: 7.0
2022-10-06 09:31:08.334204: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 1
2022-10-06 09:31:08.334270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/repli

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/tas

In [13]:
model_gpu.predict(np.concatenate([X_test_scaled, X_test_scaled, X_test_scaled, X_test_scaled, X_test_scaled])) # We multiplied the X_test to check GPU usage

1563/1563 [==============================] - 18s 11ms/step


array([[3.9844909e-03, 2.0822229e-04, 1.4139652e-02, ..., 5.5011438e-04,
        7.4595697e-03, 7.5909849e-03],
       [7.7995681e-03, 7.6944524e-01, 3.5332184e-06, ..., 9.9429064e-07,
        1.8379286e-01, 3.8945921e-02],
       [7.0480712e-02, 1.6722994e-01, 2.8469465e-03, ..., 7.3819066e-04,
        5.4288960e-01, 2.0992452e-01],
       ...,
       [8.9241883e-05, 9.0138319e-06, 8.3996855e-02, ..., 4.2369706e-03,
        2.7003358e-04, 6.4079111e-05],
       [2.8460411e-02, 5.3601044e-01, 1.8379822e-02, ..., 1.3022214e-04,
        9.0050288e-03, 2.1574055e-03],
       [6.6487199e-05, 5.2376394e-04, 3.0200728e-03, ..., 9.1520762e-01,
        3.6907738e-06, 1.1142212e-03]], dtype=float32)